# 🔬 HBV 그룹 비교 분석 - Real Data (GSE182159)

**Author**: Youngmin (Pranja)

**목적**: 실제 GSE182159 데이터로 HBV Stage별 면역세포 분포 비교

---

## 📊 HBV Stages (GSE182159)

| Stage | 샘플 수 | 설명 |
|-------|---------|------|
| **NL** | 11 | **Normal (건강인 대조군)** |
| IT | 13 | Immune Tolerant (면역 관용기) |
| IA | 9 | Immune Active (면역 활성기) |
| AR | 7 | Acute Resolved |
| AC | 6 | Acute Chronic |

## 📊 비교 분석 구조

1. **NL vs 각 보균자군** (대조군 비교)
2. **보균자군 vs 보균자군** (Pairwise)
3. **NL 대비 차이 크기 비교** (Effect size)

⚠️ **사전 조건**: `Step1_HBV_real.ipynb` 실행 완료 (243K full 분석)

In [ ]:
# @title STEP 1: 환경 설정 및 데이터 로드 (243K Full)
import os, sys
from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/tahoe-x1'
ANALYSIS_PATH = f'{PROJECT_PATH}/viral_hepatitis/analysis_results'
OUTPUT_PATH = f'{PROJECT_PATH}/viral_hepatitis/comparison_results_real'
os.makedirs(OUTPUT_PATH, exist_ok=True)

!pip install scanpy anndata scipy statsmodels seaborn mygene -q

import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multitest import multipletests
import itertools

# 실제 데이터 결과 로드 (FULL 243K 우선)
print('📊 실제 데이터 분석 결과 검색 중...')
h5ad_files = [f for f in os.listdir(ANALYSIS_PATH) if 'real' in f and f.endswith('.h5ad')]

# FULL 버전 우선 선택
full_files = [f for f in h5ad_files if 'full' in f]
if full_files:
    latest_file = sorted(full_files)[-1]
    print(f'   ✅ Full 243K 버전 발견: {latest_file}')
else:
    latest_file = sorted(h5ad_files)[-1]
    print(f'   ⚠️ Full 버전 없음, 사용: {latest_file}')

print(f'\n📊 데이터 로딩 중...')
adata = sc.read_h5ad(f'{ANALYSIS_PATH}/{latest_file}')

# Stage 컬럼 확인
stage_col = None
for col in ['hbv_stage', 'Stage', 'stage']:
    if col in adata.obs.columns:
        stage_col = col
        break

# tissue 컬럼 확인/생성
if 'tissue' not in adata.obs.columns and 'sample' in adata.obs.columns:
    adata.obs['tissue'] = adata.obs['sample'].apply(
        lambda x: 'Blood' if 'Blood' in str(x) else ('Liver' if 'Liver' in str(x) else 'Unknown')
    )

print(f'\n✅ STEP 1 완료!')
print(f'   세포 수: {adata.n_obs:,} {"(Full 243K)" if adata.n_obs > 200000 else "(Sampled)"}')
print(f'   Stage 컬럼: {stage_col}')

print(f'\n📊 HBV Stage 분포:')
stage_counts = adata.obs[stage_col].value_counts()
print(stage_counts)

# NL (대조군) 확인
if 'NL' in stage_counts.index:
    print(f'\n✅ NL (대조군) 확인: {stage_counts["NL"]:,} cells')
else:
    print('⚠️ NL 대조군이 없습니다!')

if 'tissue' in adata.obs.columns:
    print(f'\n📊 조직별 분포:')
    print(adata.obs['tissue'].value_counts())

In [ ]:
# @title STEP 2: 전체 시각화 (UMAP)
print('📊 UMAP 시각화')

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sc.pl.umap(adata, color='cell_type', ax=axes[0], show=False,
           title='Cell Types', legend_loc='on data', legend_fontsize=7)

sc.pl.umap(adata, color=stage_col, ax=axes[1], show=False,
           title=f'HBV Stage (NL = Control)')

if 'tissue' in adata.obs.columns:
    sc.pl.umap(adata, color='tissue', ax=axes[2], show=False,
               title='Tissue')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/umap_overview.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 2 완료!')

In [ ]:
# @title STEP 3: 세포 유형별 비율 분석
print('📊 세포 유형별 비율 분석')

stages = sorted(adata.obs[stage_col].dropna().unique())
cell_types = sorted(adata.obs['cell_type'].unique())

# 비율 계산
cell_type_counts = adata.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
cell_type_ratios = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0) * 100

print('\n📋 Stage별 세포 유형 비율 (%):')
print(cell_type_ratios.round(1))

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# NL을 맨 앞으로
stage_order = ['NL'] + [s for s in stages if s != 'NL']
cell_type_ratios_ordered = cell_type_ratios.reindex(stage_order)

cell_type_ratios_ordered.plot(kind='bar', stacked=True, ax=axes[0], colormap='tab10')
axes[0].set_title('Cell Type Distribution by Stage (NL = Control)', fontsize=12)
axes[0].set_xlabel('HBV Stage')
axes[0].set_ylabel('Percentage (%)')
axes[0].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
axes[0].axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='Control')

sns.heatmap(cell_type_ratios_ordered.T, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[1])
axes[1].set_title('Cell Type Percentage Heatmap', fontsize=12)

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/cell_type_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 3 완료!')

In [ ]:
# @title STEP 4: NL (대조군) vs 각 보균자군 비교 ⭐
print('📊 NL (대조군) vs 각 보균자군 비교')
print('='*70)

# NL 데이터
nl_cells = adata[adata.obs[stage_col] == 'NL']
nl_ratios = {}
for ct in cell_types:
    nl_ratios[ct] = (nl_cells.obs['cell_type'] == ct).mean() * 100

print(f'\n📋 NL (대조군) 세포 비율:')
for ct, ratio in nl_ratios.items():
    print(f'   {ct}: {ratio:.2f}%')

# 각 보균자군과 NL 비교
print('\n' + '='*70)
print('📋 NL vs 각 보균자군 비교')
print('='*70)

nl_comparison = []
other_stages = [s for s in stages if s != 'NL']

for stage in other_stages:
    stage_cells = adata[adata.obs[stage_col] == stage]
    
    # Chi-square
    subset = adata[adata.obs[stage_col].isin(['NL', stage])]
    cont = subset.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
    chi2, p_val, _, _ = stats.chi2_contingency(cont)
    
    row = {
        'Comparison': f'NL vs {stage}',
        'NL cells': len(nl_cells),
        f'{stage} cells': len(stage_cells),
        'Chi2': round(chi2, 2),
        'p-value': p_val,
        'Significant': '✅' if p_val < 0.05 else ''
    }
    nl_comparison.append(row)

nl_vs_df = pd.DataFrame(nl_comparison)

# FDR 보정
if len(nl_vs_df) > 0:
    _, p_adj, _, _ = multipletests(nl_vs_df['p-value'], method='fdr_bh')
    nl_vs_df['p-adj (FDR)'] = p_adj
    nl_vs_df['FDR Sig'] = ['✅' if p < 0.05 else '' for p in p_adj]

print(nl_vs_df.to_string(index=False))
nl_vs_df.to_csv(f'{OUTPUT_PATH}/NL_vs_each_stage.csv', index=False)

print(f'\n✅ STEP 4 완료!')

In [ ]:
# @title STEP 5: NL 대비 세포 유형별 차이 (Effect Size) ⭐
print('📊 NL 대비 세포 유형별 비율 차이 (Effect Size)')
print('='*70)

effect_results = []

for ct in cell_types:
    row = {'Cell Type': ct, 'NL (%)': round(nl_ratios[ct], 2)}
    
    for stage in other_stages:
        stage_cells = adata[adata.obs[stage_col] == stage]
        stage_ratio = (stage_cells.obs['cell_type'] == ct).mean() * 100
        diff = stage_ratio - nl_ratios[ct]
        
        row[f'{stage} (%)'] = round(stage_ratio, 2)
        row[f'{stage} Δ'] = round(diff, 2)  # NL 대비 차이
    
    effect_results.append(row)

effect_df = pd.DataFrame(effect_results)
print(effect_df.to_string(index=False))
effect_df.to_csv(f'{OUTPUT_PATH}/NL_effect_size.csv', index=False)

# 시각화: NL 대비 차이 히트맵
diff_cols = [c for c in effect_df.columns if 'Δ' in c]
if diff_cols:
    diff_data = effect_df.set_index('Cell Type')[diff_cols]
    diff_data.columns = [c.replace(' Δ', '') for c in diff_cols]
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(diff_data, annot=True, fmt='.1f', cmap='RdBu_r', center=0,
                cbar_kws={'label': 'Difference from NL (%)'})
    plt.title('Cell Type Proportion Difference from NL (Control)', fontsize=12)
    plt.xlabel('HBV Stage')
    plt.ylabel('Cell Type')
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_PATH}/NL_effect_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()

print(f'\n✅ STEP 5 완료!')

In [ ]:
# @title STEP 6: 보균자군 간 Pairwise 비교
print('📊 보균자군 간 Pairwise 비교 (NL 제외)')
print('='*70)

pairwise_results = []

# 모든 Stage 쌍 비교 (NL 포함)
for s1, s2 in itertools.combinations(stages, 2):
    subset = adata[adata.obs[stage_col].isin([s1, s2])]
    cont = subset.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
    
    if len(cont) == 2:
        chi2, p_val, _, _ = stats.chi2_contingency(cont)
        
        # NL 포함 여부 표시
        is_nl_comparison = 'NL' in [s1, s2]
        
        pairwise_results.append({
            'Comparison': f'{s1} vs {s2}',
            'Type': 'NL comparison' if is_nl_comparison else 'Between stages',
            'Chi2': round(chi2, 2),
            'p-value': p_val,
            'Sig': '✅' if p_val < 0.05 else ''
        })

pairwise_df = pd.DataFrame(pairwise_results)

# FDR 보정
if len(pairwise_df) > 0:
    _, p_adj, _, _ = multipletests(pairwise_df['p-value'], method='fdr_bh')
    pairwise_df['p-adj (FDR)'] = p_adj
    pairwise_df['FDR Sig'] = ['✅' if p < 0.05 else '' for p in p_adj]

print('\n📋 모든 Pairwise 비교:')
print(pairwise_df.to_string(index=False))
pairwise_df.to_csv(f'{OUTPUT_PATH}/pairwise_all.csv', index=False)

# NL 비교 vs 보균자군 간 비교 분리
print('\n' + '='*70)
print('📋 요약: NL 비교 vs 보균자군 간 비교')
print('='*70)

nl_comparisons = pairwise_df[pairwise_df['Type'] == 'NL comparison']
between_comparisons = pairwise_df[pairwise_df['Type'] == 'Between stages']

print(f'\n🔵 NL vs 보균자군: {len(nl_comparisons)}개 비교')
print(f'   유의한 비교: {(nl_comparisons["Sig"] == "✅").sum()} / {len(nl_comparisons)}')

print(f'\n🟢 보균자군 간: {len(between_comparisons)}개 비교')
print(f'   유의한 비교: {(between_comparisons["Sig"] == "✅").sum()} / {len(between_comparisons)}')

print(f'\n✅ STEP 6 완료!')

In [ ]:
# @title STEP 7: 세포 유형별 Stage 간 유의성 검정
print('📊 세포 유형별 Stage 간 유의성 검정')
print('='*70)

celltype_stats = []

for ct in cell_types:
    row = {'Cell Type': ct}
    
    # NL 비율
    row['NL (%)'] = round(nl_ratios[ct], 2)
    
    # 각 Stage 비율
    max_ratio, min_ratio = nl_ratios[ct], nl_ratios[ct]
    for stage in other_stages:
        s_cells = adata[adata.obs[stage_col] == stage]
        ratio = (s_cells.obs['cell_type'] == ct).mean() * 100
        row[f'{stage} (%)'] = round(ratio, 2)
        max_ratio = max(max_ratio, ratio)
        min_ratio = min(min_ratio, ratio)
    
    row['Max-Min'] = round(max_ratio - min_ratio, 2)
    
    # Chi-square (이 세포 유형에 대해 모든 Stage 간)
    ct_binary = (adata.obs['cell_type'] == ct).astype(int)
    cont = pd.crosstab(adata.obs[stage_col], ct_binary)
    if cont.shape[1] == 2:
        chi2, p_val, _, _ = stats.chi2_contingency(cont)
        row['p-value'] = f'{p_val:.2e}'
        row['Sig'] = '✅' if p_val < 0.05 else ''
    
    celltype_stats.append(row)

celltype_df = pd.DataFrame(celltype_stats)
print(celltype_df.to_string(index=False))
celltype_df.to_csv(f'{OUTPUT_PATH}/celltype_statistics.csv', index=False)

print(f'\n✅ STEP 7 완료!')

In [ ]:
# @title STEP 8: 조직별 분석 (Blood vs Liver)
if 'tissue' not in adata.obs.columns:
    print('⚠️ tissue 컬럼이 없어 건너뜁니다.')
else:
    print('📊 조직별 분석 (Blood vs Liver)')
    
    # Blood vs Liver 세포 유형 비교
    tissue_comparison = []
    
    for ct in cell_types:
        blood = adata[adata.obs['tissue'] == 'Blood']
        liver = adata[adata.obs['tissue'] == 'Liver']
        
        blood_ratio = (blood.obs['cell_type'] == ct).mean() * 100 if len(blood) > 0 else 0
        liver_ratio = (liver.obs['cell_type'] == ct).mean() * 100 if len(liver) > 0 else 0
        
        # Fisher's exact
        blood_ct = (blood.obs['cell_type'] == ct).sum()
        blood_other = len(blood) - blood_ct
        liver_ct = (liver.obs['cell_type'] == ct).sum()
        liver_other = len(liver) - liver_ct
        
        if blood_other > 0 and liver_other > 0:
            _, p_val = stats.fisher_exact([[blood_ct, blood_other], [liver_ct, liver_other]])
        else:
            p_val = 1.0
        
        tissue_comparison.append({
            'Cell Type': ct,
            'Blood (%)': round(blood_ratio, 2),
            'Liver (%)': round(liver_ratio, 2),
            'Diff': round(blood_ratio - liver_ratio, 2),
            'p-value': f'{p_val:.2e}',
            'Sig': '✅' if p_val < 0.05 else ''
        })
    
    tissue_df = pd.DataFrame(tissue_comparison)
    print(tissue_df.to_string(index=False))
    tissue_df.to_csv(f'{OUTPUT_PATH}/tissue_comparison.csv', index=False)
    
    print(f'\n✅ STEP 8 완료!')

In [ ]:
# @title STEP 9: 종합 결과 저장
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 통계 요약
n_sig_nl = (nl_vs_df['Significant'] == '✅').sum() if 'nl_vs_df' in dir() else 0
n_sig_pairwise = (pairwise_df['Sig'] == '✅').sum() if 'pairwise_df' in dir() else 0
n_sig_celltype = (celltype_df['Sig'] == '✅').sum() if 'celltype_df' in dir() else 0
n_sig_tissue = (tissue_df['Sig'] == '✅').sum() if 'tissue_df' in dir() else 0

summary = f"""
================================================================================
🦠 HBV 실제 데이터 (GSE182159) 비교 분석 결과
================================================================================
Author: Youngmin (Pranja)
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

================================================================================
📊 데이터 요약
================================================================================
   - 세포 수: {adata.n_obs:,}
   - 유전자 수: {adata.n_vars:,}
   - HBV Stage: {stages}
   - 세포 유형: {len(cell_types)}개
   - 대조군 (NL): {len(nl_cells):,} cells

📋 HBV Stage 분포:
{adata.obs[stage_col].value_counts().to_string()}

================================================================================
🔬 통계 결과 요약
================================================================================

1️⃣ NL (대조군) vs 각 보균자군:
   - 비교 수: {len(other_stages)}
   - 유의한 비교: {n_sig_nl} / {len(other_stages)}

2️⃣ 모든 Pairwise 비교:
   - 총 비교 쌍: {len(pairwise_df)}
   - 유의한 쌍 (p<0.05): {n_sig_pairwise}

3️⃣ 세포 유형별 Stage 간 차이:
   - 유의한 세포 유형: {n_sig_celltype} / {len(cell_types)}

4️⃣ Blood vs Liver:
   - 유의한 세포 유형: {n_sig_tissue} / {len(cell_types)}

================================================================================
📁 저장된 파일
================================================================================
- umap_overview.png
- cell_type_distribution.png
- NL_effect_heatmap.png
- NL_vs_each_stage.csv (NL vs 각 보균자군)
- NL_effect_size.csv (NL 대비 차이)
- pairwise_all.csv (모든 Pairwise)
- celltype_statistics.csv
- tissue_comparison.csv

================================================================================
"""

print(summary)

with open(f'{OUTPUT_PATH}/summary_{timestamp}.txt', 'w', encoding='utf-8') as f:
    f.write(summary)

print('\n🎉 분석 완료!')
print(f'📁 결과 폴더: {OUTPUT_PATH}')